In [1]:
import numpy as np
import random
from collections import deque
import gym

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from keras.models import  clone_model

C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\flatbuffers\compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\keras\utils\image_utils.py:36: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\keras\utils\image_utils.py:37: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\keras\utils\image_utils.py:38: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
C:\Users\pc\anaconda3\envs\my_rl_env\lib\site

In [3]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\gym\utils\passive_env_checker.py:98: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  "We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) "


In [4]:
env.reset()
"""" pour l'erreur de display
for step in range(1):
    env.render()
    random_action = env.action_space.sample()
    env.step(random_action)
    

env.close()
""""

C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\gym\envs\classic_control\cartpole.py:169: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  "You are calling 'step()' even though this "


In [5]:
num_observations = env.observation_space.shape[0]
num_observations

4

In [6]:
num_action = env.action_space.n
num_action

2

In [7]:
model = Sequential()

model.add(Dense(16, input_shape =(1,num_observations)))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(num_action))
model.add(Activation('linear'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 16)             80        
                                                                 
 activation (Activation)     (None, 1, 16)             0         
                                                                 
 dense_1 (Dense)             (None, 1, 32)             544       
                                                                 
 activation_1 (Activation)   (None, 1, 32)             0         
                                                                 
 dense_2 (Dense)             (None, 1, 2)              66        
                                                                 
 activation_2 (Activation)   (None, 1, 2)              0         
                                                                 
Total params: 690
Trainable params: 690
Non-trainable pa

In [9]:
#from keras.models import  clone_model
target_model = clone_model(model)

In [10]:
EPOCHS = 1000
BATCH_SIZE = 32

epsilon = 1.0
EPSILON_REDUCE = 0.995

LEARNING_RATE = 0.001
GAMMA = 0.95

In [11]:
def epsilon_greedy_action_selection(model, epsilon, observation):
    
    if np.random.random() > epsilon:
        prediction = model.predict(observation.reshape((1,1,4)))
        action = np.argmax(prediction)
    else:
        action = np.random.randint(0,env.action_space.n)
        
    return action

In [12]:
deque_1 = deque(maxlen=5)

In [13]:
deque_2 = deque(maxlen=5)

In [14]:
replay_buffer = deque(maxlen=20000)
update_target_model = 10

In [15]:
test_tuple = [(1,2,3),(4,5,6),(7,8,9)]
zipped_list = list(zip(*test_tuple))
a,b,c = zipped_list

In [16]:
def replay(replay_buffer, batch_size, model, target_model):
    
    if len(replay_buffer)< batch_size:
        return
    
    samples = random.sample(replay_buffer, batch_size)
    
    target_batch = []
    
    zipped_samples = list(zip(*samples))
    
    states, actions, rewards, new_states, dones = zipped_samples
    
    targets = target_model.predict(np.array(states))
    
    q_values = model.predict(np.array(new_states))
    
    for i in range(batch_size):
        
        q_value = max(q_values[i][0])
        
        target = targets[i].copy()
        
        if dones[i]:
            target[0][actions[i]] = rewards[i]
            
        else:
            target[0][actions[i]] = rewards[i]+q_value*GAMMA
            
        target_batch.append(target)
        
    model.fit(np.array(states), np.array(target_batch), epochs = 1, verbose = 0)

In [17]:
def update_model_handler(epoch, update_target_model, model, target_model):
    
    if epoch > 0 and epoch % update_target_model ==0:
        target_model.set_weights(model.get_weights())

In [18]:
model.compile(loss = 'mse', optimizer = (Adam(lr=LEARNING_RATE)))

C:\Users\pc\anaconda3\envs\my_rl_env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [19]:
#model.summary()

In [20]:
best_so_far = 0

for epoch in range(EPOCHS):
    
    observation = env.reset()
    
    observation = observation.reshape([1,4])
    done = False
    
    points = 0
    
    while not done:
        
        action = epsilon_greedy_action_selection(model, epsilon, observation)
        
        next_observation, reward, done, info = env.step(action)
        next_observation = next_observation.reshape([1,4])
        
        replay_buffer.append((observation, action, reward, next_observation, done))
                             
        observation = next_observation
        points +=1
        
        replay(replay_buffer, BATCH_SIZE, model, target_model)

    epsilon *= EPSILON_REDUCE
    
    update_model_handler(epoch, update_target_model, model, target_model)
    
    if points > best_so_far:
        best_so_far = points
    if epoch%25 == 0:
        print(str(epoch)+ " : Points : " +str(points)+" Epsilon : " + str(epsilon) + " BSF : "+ str(best_so_far))

0 : Points : 13 Epsilon : 0.995 BSF : 13
1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


...

1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 12ms/step


1/1 [==============================] - 0s 12ms/step


KeyboardInterrupt: 

In [21]:
epoch

149

In [22]:
observation = env.reset()

for step in range(300):
    env.render()
    
    action = np.argmax(model.predict(observation.reshape([1,4])))
    observation, reward, done, info = env.step(action)
    
    if done:
        break
        
env.close()

error: display Surface quit

In [ ]:
#intuition replay buffer
""""
deque_1 = deque(maxlen=5)

deque_1
for i in range(5):
    deque_1.append(i)
    
deque_1
deque_1.append(5)

deque_1

#_-------------------------------
deque_2 = deque(maxlen = 5)

for i in range(10):
    deque_2.append(i)
    print(deque_2)
"""